In [1]:
from io import BytesIO
import base64

import numpy
from PIL import Image as PilImage

from bokeh.plotting import figure
from bokeh.models.tools import TapTool, HoverTool, WheelZoomTool, PanTool
from bokeh import events
from bokeh.models.callbacks import CustomJS
from bokeh.io import output_file, save, show
from bokeh.models.markers import Cross, Square, SquareCross, CircleCross
from bokeh.models.sources import ColumnDataSource
from bokeh.models.widgets import Div
from bokeh.layouts import widgetbox, row
from bokeh.io import output_notebook
output_notebook()

img_data = '...'

#
# Image
#
#o_img = PilImage.open('1143_Alob_2015_05_18_001.jpg').convert('RGBA')
o_img = PilImage.open(BytesIO(base64.b64decode(img_data))).convert('RGBA')
xdim, ydim = o_img.size
img = numpy.empty((ydim, xdim), dtype=numpy.uint32)
view = img.view(dtype=numpy.uint8).reshape((ydim, xdim, 4))
# Copy the RGBA image into view, flipping it so it comes right-side up
# with a lower-left origin
view[:,:,:] = numpy.flipud(numpy.asarray(o_img))

#
# Display the 32-bit RGBA image
#
dim = max(xdim, ydim)
plot = figure(plot_width=600, plot_height=500,
              x_range=[0,xdim], y_range=[0,ydim],
              tools='', logo=None,
              title='Bokeh Frog')

plot.image_rgba(image=[img], x=0, y=0, dw=xdim, dh=ydim)

#
# Data Source holding the Point Coordinates
#
source = ColumnDataSource(data=dict(x=[240, 242, 50], y=[97, 158, 120], db_id=[12, 13, 99]))
selection_source = ColumnDataSource(data=dict(db_id=[]))

#
# Double Click for Point Creation
#
dtap_cb = CustomJS(args=dict(source=source),
                    code="""
// the event that triggered the callback is cb_obj:
console.log('DoubleTap');
var data = source.data;
data['x'].push(cb_obj.x);
data['y'].push(cb_obj.y);
data['db_id'].push(Math.max.apply(null, data['db_id'])+1);
source.change.emit();
""")
plot.js_on_event(events.DoubleTap, dtap_cb)

#
# Show the position of the Point when moving selected point
#
move_cb = CustomJS(args=dict(source=source, selection_source=selection_source),
                    code="""
if (selection_source.data['db_id'].length & source.selected['1d'].indices.length) {
  console.log('Mouse Move');
  var index = source.selected['1d'].indices[0];
  var data = source.data;
  data['x'][index] = cb_obj.x;
  data['y'][index] = cb_obj.y;
  source.change.emit();
}
""")
plot.js_on_event(events.MouseMove, move_cb)

#
# Select and Release Point on click
# is a little bit hacky
#
sg_cb = CustomJS(args=dict(source=source, selection_source=selection_source),
                    code="""
console.log('SelectionGeometry');
// nothing selected
if (source.selected['1d'].indices.length) {
    if (selection_source.data['db_id'].length) {
        console.log('Clear selection');
        selection_source.data['db_id'].length = 0;
        source.selection_manager.clear();
        source.change.emit();
    }
    else {
        console.log('Add new selection');
        selection_source.data['db_id'].length=0;
        selection_source.data['db_id'].push(source.selected['1d'].indices[0]);
        selection_source.change.emit();
    }
}
""")
plot.js_on_event(events.SelectionGeometry, sg_cb)

#
# Glyph
#
GLYPH_CLASS = CircleCross#Square# Circle, Cross
FILL_ALPHA = 0
LINE_WIDTH = 1.2
LINE_COLOR = 'yellow'
glyph = GLYPH_CLASS(size=36, x='x', y='y',
                    line_width=LINE_WIDTH, line_color=LINE_COLOR,
                    fill_alpha=FILL_ALPHA)
cr = plot.add_glyph(source, glyph)
#
# Define Selection and Hover of Glyph
#
cr.selection_glyph = GLYPH_CLASS(line_color='firebrick', line_width=LINE_WIDTH,
                                 fill_alpha=FILL_ALPHA)
cr.nonselection_glyph = GLYPH_CLASS(line_width=LINE_WIDTH, line_color=LINE_COLOR,
                                    fill_alpha=FILL_ALPHA)
cr.hover_glyph = GLYPH_CLASS(line_color='firebrick', line_width=LINE_WIDTH,
                             fill_alpha=FILL_ALPHA)

#
# Plot Tools
#

# The TapTool is needed to get the selection
plot.add_tools(TapTool(renderers=[cr]))
# Hover to inform the user that the glyph can be selected
plot.add_tools(HoverTool(tooltips=None, renderers=[cr]))
# Scroll
wz = WheelZoomTool()
plot.add_tools(wz)
plot.toolbar.active_scroll = wz
pt = PanTool()
plot.add_tools(pt)

#
# Doc Div
#
p = Div(text='''
<h2>HowTo Label the Frog</h2>
<font style="font-size: 1.1em">
<dl>
  <dt>New Point:</dt><dd>Double-Click</dd>
  <dt>Edit Point:</dt><dd>Click (Select) > Drag > Click (Place)</dd>
  <dt>Delete Point:</dt><dd><b color="red">Not Working</b></dd>
</dl>
<p>Please label all my warts.</p>
<p>And don't call me frog, I'm a <b>toad</b>!</p>
</font>
''')

#
# Layout
#
layout = row(widgetbox(p), plot)

#output_file('bokeh_frog.html')
#save(layout)
r = show(layout, notebook_handle=True)




Loading BokehJS ...